# MVP ENGENHARIA DE DADOS - ANÁLISE EXPLORATÓRIA (SQL)

**Autor:** Thiago Maciel Barbosa  
**Objetivo:** Responder às perguntas de negócio e validar a qualidade dos dados.

---

In [0]:
-- -----------------------------------------------------------------------------
-- 1. ANÁLISE DE CONSERVAÇÃO POR FAMÍLIA
-- Pergunta: Quais famílias possuem a maior proporção de espécies ameaçadas?
-- -----------------------------------------------------------------------------
use birdbase; 

SELECT 
    t.familia,
    -- Contagem total de espécies na família
    COUNT(DISTINCT f.sk_especie) as total_especies,
    -- Contagem apenas das ameaçadas (CR, EN, VU)
    SUM(CASE WHEN c.cod_categoria_iucn IN ('VU', 'EN', 'CR') THEN 1 ELSE 0 END) as total_ameacadas,
    -- Cálculo da porcentagem
    ROUND(
        (SUM(CASE WHEN c.cod_categoria_iucn IN ('VU', 'EN', 'CR') THEN 1 ELSE 0 END) / COUNT(DISTINCT f.sk_especie)) * 100
    , 2) as pct_ameaca
FROM fato_metricas_aves f
JOIN dim_taxonomia t ON f.sk_familia = t.sk_familia
JOIN dim_conservacao c ON f.sk_status_iucn = c.sk_status_iucn
GROUP BY t.familia
HAVING total_especies > 20 -- Filtramos famílias pequenas para evitar distorção estatística
ORDER BY pct_ameaca DESC
LIMIT 15;

familia,total_especies,total_ameacadas,pct_ameaca
Diomedeidae,21,17,80.95
Cacatuidae,22,14,63.64
Procellariidae,95,51,53.68
Cracidae,55,24,43.64
Bucerotidae,58,25,43.1
Megapodiidae,21,9,42.86
Otididae,26,11,42.31
Phalacrocoracidae,34,12,35.29
Acrocephalidae,54,18,33.33
Psittacidae,172,56,32.56


Databricks visualization. Run in Databricks to view.

In [0]:
-- -----------------------------------------------------------------------------
-- 2. RELAÇÃO DIETA vs. MASSA CORPORAL
-- Pergunta: A dieta influencia no tamanho médio das aves? Carnívoros são maiores?
-- -----------------------------------------------------------------------------

SELECT 
    d.dieta_primaria,
    ROUND(AVG(f.massa_media), 2) as massa_media_gramas,
    COUNT(DISTINCT f.sk_especie) as qtd_especies
FROM fato_metricas_aves f
JOIN dim_dieta d ON f.sk_dieta = d.sk_dieta
WHERE f.massa_media > 0 
  AND d.dieta_primaria IS NOT NULL 
  AND d.dieta_primaria != ''
GROUP BY d.dieta_primaria
ORDER BY massa_media_gramas DESC;


dieta_primaria,massa_media_gramas,qtd_especies
SCAVENGER,5041.13,28
PLANT,1721.29,154
FISH,1455.12,274
HERBIVORE,995.88,389
VERTEBRATE,769.05,313
CARNIVORE,652.74,201
OMNIVORE,373.11,1016
FRUIT,373.03,1194
NO INFORMATION,312.36,5
INVERTEBRATE,102.68,5657


Databricks visualization. Run in Databricks to view.

In [0]:
-- -----------------------------------------------------------------------------
-- 3. HABITAT DAS ESPÉCIES AMEAÇADAS
-- Pergunta: Quais habitats concentram o maior número de espécies em perigo crítico?
-- -----------------------------------------------------------------------------

SELECT 
    h.habitat_primario,
    COUNT(DISTINCT f.sk_especie) as contagem_especies_em_perigo
FROM fato_metricas_aves f
JOIN dim_habitat h ON f.sk_habitat = h.sk_habitat
JOIN dim_conservacao c ON f.sk_status_iucn = c.sk_status_iucn
WHERE c.cod_categoria_iucn IN ('CR', 'EN') -- Apenas Crítico e Em Perigo
  AND h.habitat_primario IS NOT NULL 
  AND h.habitat_primario != ''
GROUP BY h.habitat_primario
ORDER BY contagem_especies_em_perigo DESC
LIMIT 10;

habitat_primario,contagem_especies_em_perigo
FOREST,358
WETLAND,49
SEA,46
GRASSLAND,37
SHRUB,33
WOODLAND,27
COASTAL,20
RIPARIAN,12
SAVANNA,7
PLAINS,6


Databricks visualization. Run in Databricks to view.

In [0]:
-- -----------------------------------------------------------------------------
-- 4. MIGRAÇÃO vs. TAMANHO DA NINHADA
-- Pergunta: Aves que migram tendem a ter ninhadas maiores ou menores?
-- -----------------------------------------------------------------------------

SELECT 
    CASE 
        WHEN eh_migratorio = true THEN 'Migratório' 
        WHEN eh_sedentario = true THEN 'Sedentário'
        ELSE 'Misto/Nômade'
    END as comportamento_migratorio,
    ROUND(AVG(ninhada_max), 2) as media_tamanho_ninhada,
    COUNT(*) as amostra
FROM fato_metricas_aves
WHERE ninhada_max > 0
GROUP BY 1
ORDER BY media_tamanho_ninhada DESC;

comportamento_migratorio,media_tamanho_ninhada,amostra
Migratório,4.96,1550
Sedentário,3.56,6272
Misto/Nômade,3.43,970


Databricks visualization. Run in Databricks to view.

In [0]:
-- -----------------------------------------------------------------------------
-- 5. QUALIDADE DE DADOS (Check Final)
-- Objetivo: Provar que o ETL não gerou dados órfãos ou inconsistentes.
-- -----------------------------------------------------------------------------

SELECT '1. Órfãos de Espécie (Fato sem Dim)' as teste, COUNT(*) as falhas 
FROM fato_metricas_aves WHERE sk_especie IS NULL
UNION ALL
SELECT '2. Órfãos de Habitat', COUNT(*) 
FROM fato_metricas_aves WHERE sk_habitat IS NULL
UNION ALL
SELECT '3. Órfãos de Taxonomia', COUNT(*) 
FROM fato_metricas_aves WHERE sk_familia IS NULL
UNION ALL
SELECT '4. Massa Negativa (Erro de Limpeza)', COUNT(*) 
FROM fato_metricas_aves WHERE massa_media < 0;

teste,falhas
1. Órfãos de Espécie (Fato sem Dim),0
2. Órfãos de Habitat,0
3. Órfãos de Taxonomia,0
4. Massa Negativa (Erro de Limpeza),0
